Run at the begining

In [ ]:
!git clone https://github.com/polimi-ispl/icpr2020dfdc
!pip install efficientnet-pytorch
!pip install -U git+https://github.com/albu/albumentations > /dev/null
%cd icpr2020dfdc/notebook
!pip install Flask torch torchvision opencv-python Pillow matplotlib pyngrok

Cloning into 'icpr2020dfdc'...
remote: Enumerating objects: 656, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 656 (delta 101), reused 87 (delta 83), pack-reused 537 (from 1)
Receiving objects: 100% (656/656), 99.64 MiB | 18.26 MiB/s, done.
Resolving deltas: 100% (341/341), done.
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=31db44e3e54735b644d1be4c2cfd732ef1ac156d8b9c53aa5d14d680bf139a48
  Stored in directory: /root/.cache/pip/wheels/8b/6f/9b/231a832f811ab6ebb1b32455b177ffc6b8b1cd8de19de70c09
Successfully built efficientnet-pytorch
  Running command git clone --filter=blob:none --quiet https://github.com/albu/albumentations /tmp/pip-req-build-qytakfcs
/content/icpr2020dfdc/notebook


Then run this :

In [ ]:
# Step 1: Import Required Libraries
from flask import Flask, request
import torch
from torch.utils.model_zoo import load_url
import cv2
import os
from pyngrok import ngrok
from PIL import Image
from werkzeug.utils import secure_filename

import sys
sys.path.append('..')

# Import your custom modules (make sure these are available in your environment)
from blazeface import FaceExtractor, BlazeFace
from architectures import fornet, weights
from isplutils import utils

app = Flask(__name__)

# Step 2: Load Your Model and Weights
net_model = 'EfficientNetAutoAttB4'
train_db = 'DFDC'

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224

model_url = weights.weight_url['{:s}_{:s}'.format(net_model,train_db)]
net = getattr(fornet,net_model)().eval().to(device)
net.load_state_dict(load_url(model_url,map_location=device,check_hash=True))

transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

facedet = BlazeFace().to(device)
facedet.load_weights("../blazeface/blazeface.pth")
facedet.load_anchors("../blazeface/anchors.npy")
face_extractor = FaceExtractor(facedet=facedet)

@app.route('/')
def index():
    return '''
    <h1>Upload an Image, Video, or GIF</h1>
    <form action="/upload" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*,video/*,.gif" required>
        <input type="submit" value="Upload">
    </form>
    '''

# Specify the upload folder
upload = '/content/uploads'
app.config['UPLOAD_FOLDER'] = upload

# Create the upload folder if it doesn't exist
if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

@app.route('/upload', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']

        # Secure the filename and save the file
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)  # Save the file
        result = None

        # Process the file based on its type
        if file.filename.endswith(('.mp4', '.avi', '.mov')):
            result = process(file_path)
        elif file.filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            result =  process(file_path)
        else:
          result = "Unsupported file type."

        # Return the result and render the upload form again

        return f'''
          <!DOCTYPE html>
          <html lang="en">
          <head>
              <meta charset="UTF-8">
              <meta name="viewport" content="width=device-width, initial-scale=1.0">
              <title>Upload File</title>
          </head>
          <body>
              <h1>Upload an Image, Video, or GIF</h1>
              <form action="/upload" method="post" enctype="multipart/form-data">
                  <input type="file" name="file" accept="image/*,video/*,.gif" required>
                  <input type="submit" value ="Upload">
              </form>

              <h2>Result:</h2>
              <p>{result}</p>
          </body>
          </html>
          '''
def process(detect_path):
    cam = cv2.VideoCapture(detect_path)
    last_frame = None

    while True:
        ret, frame = cam.read()
        if ret:
            last_frame = frame
        else:
            break

    cam.release()

    if last_frame is not None:
        return analyze_frame(last_frame)
    else:
        return "No frames detected"

def analyze_frame(frame):
    # Check if the frame is empty
    if frame is None or frame.size == 0:
        return "Error: The frame is empty."

    # Convert BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Extract face
    image_face = face_extractor.process_image(img=frame_rgb)
    if 'faces' in image_face and len(image_face['faces']) > 0:
        image_face = image_face['faces'][0]
    else:
        return "No face detected"

    # Prepare the face for prediction
    faces_t = torch.stack([transf(image=im)['image'] for im in [image_face]])

    with torch.no_grad():
        faces_pred = torch.sigmoid(net(faces_t.to(device))).cpu().numpy().flatten()

    # Print scores
    score = faces_pred[0]
    result = "Image is fake" if score > 0.5 else "Image is real"
    return f'Score for face: {score:.5f}. {result}'

# Step 3: Start the Flask App
if __name__ == '__main__':
    # Start ngrok
    ngrok.set_auth_token('2rTV3LPAsyBUa3eHsXv7V6kpGb1_25Tcwtwofie8Vi2rh1DFb')  # This is my token, but ig you can use it too
    public_url = ngrok.connect(5000)
    print(f' * ngrok tunnel "{public_url}" ')

    app.run(port=5000)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth
100%|██████████| 74.4M/74.4M [00:00<00:00, 119MB/s]
Downloading: "https://f002.backblazeb2.com/file/icpr2020/EfficientNetAutoAttB4_DFDC_bestval-72ed969b2a395fffe11a0d5bf0a635e7260ba2588c28683630d97ff7153389fc.pth" to /root/.cache/torch/hub/checkpoints/EfficientNetAutoAttB4_DFDC_bestval-72ed969b2a395fffe11a0d5bf0a635e7260ba2588c28683630d97ff7153389fc.pth


Loaded pretrained weights for efficientnet-b4


100%|██████████| 33.9M/33.9M [00:00<00:00, 51.1MB/s]


/content/icpr2020dfdc/notebook/../isplutils/utils.py:155: UserWarning: Argument 'value' is not valid and will be ignored.
  A.PadIfNeeded(min_height=patch_size, min_width=patch_size,
/content/icpr2020dfdc/notebook/../isplutils/utils.py:155: UserWarning: Argument 'always_apply' is not valid and will be ignored.
  A.PadIfNeeded(min_height=patch_size, min_width=patch_size,
/content/icpr2020dfdc/notebook/../isplutils/utils.py:157: UserWarning: Argument 'always_apply' is not valid and will be ignored.
  A.Resize(height=patch_size,width=patch_size,always_apply=True),
/content/icpr2020dfdc/notebook/../blazeface/blazeface.py:164: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the

 * ngrok tunnel "NgrokTunnel: "https://7bef-34-82-142-147.ngrok-free.app" -> "http://localhost:5000"" 
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 07:04:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2025 07:04:06] "GET /favicon.ico HTTP/1.1" 404 -
